In [3]:

import torch
import torch.distributions
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.contrib.cevae import CEVAE

In [ ]:
!pip3 install pyro-ppl

In [4]:
import os

os.chdir('/Users/gabriellethivierge/Documents/10708-project/raw_twins/')
dat = pd.read_csv("cleaned_twins.csv")
dat.drop(dat.columns[[0, 1]], axis=1, inplace=True)

In [5]:

def flip(x, p):
      s = np.random.binomial(1, p, 1)
      if s == 1:
            z = 1 - x # flip it
      else:
            z = x # don't flip it
      return(z)

gestat_cols = [col for col in dat.columns if 'gestat' in col]

twin_data = dat[gestat_cols].applymap(lambda x: flip(x, 0.2))




In [6]:
# make x and y sets

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
import random
from random import choices

twin_data['fold'] = random.choices([1,2,3,4,5,6,7,8,9,10], k=len(twin_data.index))

# twin_x = twin_data.drop(['ID', 'bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
# twin_x[['dtotord_min', 'dlivord_min']] = twin_x[['dtotord_min', 'dlivord_min']].astype('category')
# twin_y = twin_data['Y']


In [7]:

def do_cevae(dat, num_layers_nh, flip_prob):

    gestat_cols = [col for col in dat.columns if 'gestat' in col]

    dat[gestat_cols].applymap(lambda x: flip(x, flip_prob))
    twin_data = dat
    twin_data['fold'] = random.choices([1,2,3,4,5,6,7,8,9,10], k=len(twin_data.index))

    results = []
    naive = []
    lr = []
    for i in range(1, 11):
        train = twin_data[twin_data['fold'] != i]
        test = twin_data[twin_data['fold'] == i]


        x_train = train.drop(['bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
        x_train[['dtotord_min', 'dlivord_min']] = x_train[['dtotord_min', 'dlivord_min']].astype('category')

        x_test = test.drop(['bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
        x_test[['dtotord_min', 'dlivord_min']] = x_test[['dtotord_min', 'dlivord_min']].astype('category')


        x_train = torch.tensor(np.array(x_train)).to(torch.float32)
        y_train = torch.tensor(np.array(train['Y'])).to(torch.float32)
        t_train = torch.tensor(np.array(train['Treatment'])).to(torch.float32)

        x_test = torch.tensor(np.array(x_test)).to(torch.float32)
        y_test = torch.tensor(np.array(test['Y'])).to(torch.float32)
        t_test = torch.tensor(np.array(test['Treatment'])).to(torch.float32)

        cevae = CEVAE(
        feature_dim = 74,
        latent_dim  = 50,
        hidden_dim = 200,
        num_layers = num_layers_nh,
        num_samples = len(x_train),
        outcome_dist = "bernoulli",
        )

        cevae.fit(
        x_train,
        t_train,
        y_train,
        num_epochs = 100,
        batch_size = 500,
        learning_rate = 0.004,
        learning_rate_decay = 0.1,
        weight_decay = 1e-4,
        )

        naive_ate = y_test[t_test == 1].mean() - y_test[t_test == 0].mean()
        est_ite = cevae.ite(x_test)
        est_ate = est_ite.mean()

        naive.append(naive_ate)
        results.append(est_ate)

    return(naive, results)

# y_test = test['Y']

# y2 = np.array(y_train)
# x_train = torch.tensor(x2).to(torch.float32)

# ## prep for input


# t = twin_data['Treatment']
# t_train = torch.tensor(t).to(torch.float32)

# y = twin_data['Y']



In [30]:
def do_cevae(dat, num_layers_nh, flip_prob):
    gestat_cols = [col for col in dat.columns if 'gestat' in col]
    dat['fold'] = random.choices([1,2,3,4,5], k=len(dat.index))

    # flip_prob = 0.2
    # num_layers_nh = 2

    results = []
    true_ate = []
    for prob in flip_prob:
        datcp = dat.copy()
        datcp[gestat_cols].applymap(lambda x: flip(x, prob))
        twin_data = datcp
    
        i = 1
        # for i in range(1, 11):
        train = twin_data[twin_data['fold'] != i]
        test = twin_data[twin_data['fold'] == i]

        y1 = test[['mort_1']].mean()
        y0 = test[['mort_0']].mean()
        true_ate.append(y1[0] - y0[0])

        x_train = train.drop(['bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
        x_train[['dtotord_min', 'dlivord_min']] = x_train[['dtotord_min', 'dlivord_min']].astype('category')

        x_test = test.drop(['bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
        x_test[['dtotord_min', 'dlivord_min']] = x_test[['dtotord_min', 'dlivord_min']].astype('category')


        x_train = torch.tensor(np.array(x_train)).to(torch.float32)
        y_train = torch.tensor(np.array(train['Y'])).to(torch.float32)
        t_train = torch.tensor(np.array(train['Treatment'])).to(torch.float32)

        x_test = torch.tensor(np.array(x_test)).to(torch.float32)
        y_test = torch.tensor(np.array(test['Y'])).to(torch.float32)
        t_test = torch.tensor(np.array(test['Treatment'])).to(torch.float32)

        

        cevae = CEVAE(
        feature_dim = 74,
        latent_dim  = 30,
        hidden_dim = 200,
        num_layers = num_layers_nh,
        num_samples = len(x_train),
        outcome_dist = "bernoulli",
        )

        cevae.fit(
        x_train,
        t_train,
        y_train,
        num_epochs = 1000,
        batch_size = 500,
        learning_rate = 0.004,
        learning_rate_decay = 0.1,
        weight_decay = 1e-4,
        )

        est_ite = cevae.ite(x_test)
        est_ate = est_ite.mean()

        results.append(est_ate)

    return(true_ate, results)

In [22]:
gestat_cols = [col for col in dat.columns if 'gestat' in col]
flip_prob  = 0.2
dat[gestat_cols].applymap(lambda x: flip(x, flip_prob))
twin_data = dat
twin_data['fold'] = random.choices([1,2,3,4,5,6,7,8,9,10], k=len(twin_data.index))

i = 1
train = twin_data[twin_data['fold'] != i]
test = twin_data[twin_data['fold'] == i]
y1 = train[['mort_1']].mean()
y0 = train[['mort_0']].mean()
y1[0] - y0[0]

-0.02603686635944702

In [31]:
g = do_cevae(dat, 2, [0.1, 0.2, 0.3, 0.4, 0.5])

print(g[0])
print(g[1])

# print(results)
# print(naive)
# np.mean(results)

INFO 	 Training with 8 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 8 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 8 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 8 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 8 minibatches per epoch
INFO 	 Evaluating 1 minibatches


[-0.020576131687242788, -0.020576131687242788, -0.020576131687242788, -0.020576131687242788, -0.020576131687242788]
[tensor(-0.0261), tensor(-0.0009), tensor(0.0288), tensor(0.0188), tensor(0.0223)]


In [200]:
print(g[0])
print(g[1])
print(g[2])


# def cevae_function(x, y, t, k):

#     x_vars = torch.tensor(np.array(x.drop('Treatment'))).to(torch.float32) 
#     tx = torch.tensor(np.array(x['Treatment'])).to(torch.float32) 
#     Y = torch.tensor(np.array(y)).to(torch.float32)

#     cevae = CEVAE(
#     feature_dim = 74,
#     latent_dim  = 50,
#     hidden_dim = 200,
#     num_layers = 3,
#     num_samples = len(x_vars.index),
#     outcome_dist = "bernoulli",
#     )

#     cevae.fit(
#     x_vars,
#     tx,
#     Y,
#     num_epochs = 1000,
#     batch_size = 500,
#     learning_rate = 0.004,
#     learning_rate_decay = 0.1,
#     weight_decay = 1e-4,
#     )
    
#     est_ite = cevae.ite(x_test)
#     est_ate = est_ite.mean()
#     return(est_ate)

# Evaluate.
# x_test, t_test, y_test, true_ite = generate_data_gaussian()
# true_ate = true_ite.mean()
# print("true ATE = {:0.3g}".format(true_ate.item()))
# naive_ate = y_test[t_test == 1].mean() - y_test[t_test == 0].mean()
# print("naive ATE = {:0.3g}".format(naive_ate))


[tensor(-0.0506), tensor(-0.0506), tensor(-0.0506), tensor(-0.0506), tensor(-0.0506)]
[tensor(-0.0044), tensor(-0.0044), tensor(-0.0044), tensor(-0.0044), tensor(-0.0044)]
[tensor(0.0103), tensor(0.0436), tensor(0.0139), tensor(-0.0029), tensor(0.0078)]


In [ ]:
g[0]

INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches
INFO 	 Training with 9 minibatches per epoch
INFO 	 Evaluating 1 minibatches


In [ ]:
result_2_1 = do_cevae(dat, 2, 0.1)
result_2_2 = do_cevae(dat, 2, 0.2)
result_2_3 = do_cevae(dat, 2, 0.3)
result_2_4 = do_cevae(dat, 2, 0.4)
result_2_5 = do_cevae(dat, 2, 0.5)

In [133]:
print(g)

None


In [ ]:
est_ite = cevae.ite(x_test)
est_ate = est_ite.mean()
print("estimated ATE = {:0.3g}".format(est_ate.item()))

In [ ]:
gestat_cols = [col for col in dat.columns if 'gestat' in col]

dat[gestat_cols].applymap(lambda x: flip(x, 0.2))
twin_data = dat
twin_data['fold'] = random.choices([1,2,3,4,5,6,7,8,9,10], k=len(twin_data.index))
i = 1
train = twin_data[twin_data['fold'] != i]
test = twin_data[twin_data['fold'] == i]

x_train = train.drop(['bord_1', 'bord_0', 'mort_1', 'mort_0', 'dbirwt_1', 'dbirwt_0', 'Y'], axis=1)
        

print(len(x_test))